In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
#sys.path.append('/work/qdiff/mo_utils')
sys.executable


'/home/nadavg/anaconda3/envs/qdiff/bin/python'

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [4]:
#from mo_utils.utils.torch_utils import torch_to_pt,create_torch_script
import netron
from pathlib import Path

In [5]:
from qdiff import (
    QuantModel, QuantModule,QuantOp ,BaseQuantBlock,
    block_reconstruction, layer_reconstruction,
)
from qdiff.utils import resume_cali_model, get_train_samples


In [6]:
from scripts.gen_image import gen_image_from_prompt

In [7]:
import torch
from omegaconf import OmegaConf
from txt2img import load_model_from_config
from pytorch_lightning import seed_everything
from ldm.models.diffusion.plms import PLMSSampler
from qdiff.quant_layer import UniformAffineQuantizer
from qdiff.adaptive_rounding import AdaRoundQuantizer
import numpy as np
import matplotlib.pyplot as plt
from mo_utils.utils.plot_aux import plot_hist,  myim


/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/transformers/modeling_utils.py:371: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly 

libtmux not installed ??


In [8]:
from scripts.get_stats import gen_stats

In [ ]:
SD =  gen_stats(ddim_steps=50,num_samples=3)

/home/nadavg/q-diffusion/scripts/txt2img.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pl_sd = torch.load(ckpt, map_location="cpu")


LatentDiffusion: Running in eps-prediction mode


/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/transformers/modeling_utils.py:371: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly 


 ############# Processing timestep 0 ############# 


 ############# Processing timestep 1 ############# 


 ############# Processing timestep 2 ############# 


 ############# Processing timestep 3 ############# 


 ############# Processing timestep 4 ############# 


 ############# Processing timestep 5 ############# 


 ############# Processing timestep 6 ############# 


 ############# Processing timestep 7 ############# 


 ############# Processing timestep 8 ############# 


 ############# Processing timestep 9 ############# 


 ############# Processing timestep 10 ############# 


 ############# Processing timestep 11 ############# 


 ############# Processing timestep 12 ############# 


 ############# Processing timestep 13 ############# 


 ############# Processing timestep 14 ############# 


 ############# Processing timestep 15 ############# 


 ############# Processing timestep 16 ############# 


 ############# Processing timestep 17 ############# 


 ############# Proc

In [19]:
SD.keys()
SD['input_blocks.0'][0],SD['input_blocks.0'][1]

({'mean': tensor([[ 0.0106,  0.0002, -0.0099,  0.0119]]),
  'var': tensor([[0.9916, 0.9943, 1.0031, 1.0095]]),
  'min': tensor([[-3.6289, -3.9269, -3.6628, -3.5696]]),
  'max': tensor([[3.7151, 3.9313, 3.5423, 3.8742]]),
  'num_elements': 24576},
 {'mean': tensor([[ 0.0117, -0.0011, -0.0088,  0.0132]]),
  'var': tensor([[0.9912, 0.9934, 1.0019, 1.0082]]),
  'min': tensor([[-3.6177, -3.9200, -3.6666, -3.5637]]),
  'max': tensor([[3.7132, 3.9212, 3.5368, 3.8785]]),
  'num_elements': 24576})

In [8]:
config = OmegaConf.load(f'{Path.home()}/q-diffusion/configs/stable-diffusion/v1-inference.yaml')
model = load_model_from_config(config, "/fastdata/users/nadavg/sd/qdiff/sd-v1-4.ckpt")
device = torch.device("cuda")
model = model.to(device)
sampler = PLMSSampler(model)

/home/nadavg/q-diffusion/scripts/txt2img.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pl_sd = torch.load(ckpt, map_location="cpu")


LatentDiffusion: Running in eps-prediction mode


/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/transformers/modeling_utils.py:371: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly 

In [9]:
import gc
gc.collect()

94

In [13]:
#setattr(sampler.model.model.diffusion_model, "split", True)

In [15]:
type(sampler),type(model),type(model.model),type(model.model.diffusion_model)

(ldm.models.diffusion.plms.PLMSSampler,
 ldm.models.diffusion.ddpm.LatentDiffusion,
 ldm.models.diffusion.ddpm.DiffusionWrapper,
 ldm.modules.diffusionmodules.openaimodel.UNetModel)

In [121]:
unet = sampler.model.model.diffusion_model
from src.utils.torch_utils import add_full_name_to_module
add_full_name_to_module(unet)

In [ ]:
[b.full_name for b in unet.output_blocks]

In [10]:
np.histogram

<function numpy.histogram(a, bins=10, range=None, density=None, weights=None)>

In [144]:
class DynamicStats:
    def __init__(self, per_channel=True, device="cpu",num_channels = -1,name = None):
      
        self.device = torch.device(device)
        self.per_channel = per_channel 
        self.num_channels = num_channels
        self.name = name

        self.stats ={
            "mean": None,
            "var": None,
            "min": None,
            "max": None,
            "num_elements": 0,
        }

        # Define histogram bins
    @staticmethod
    def update(layer, data):
        self = layer.stats
        if isinstance(data, tuple):
            data = data[0]
        # n,c,h,w  > c*h*w ,n        
        N,C,H,W = data.size()
        num_elements = N*H*W
        data = data.to(self.device).permute(1, 0, 2, 3).contiguous().view(C, N * H * W)
        min_vals = data.min(dim=1)[0].reshape(1,-1)
        max_vals = data.max(dim=1)[0].reshape(1,-1)
        mean_vals = data.mean(dim=1).reshape(1,-1)
        var_vals = data.var(dim=1).reshape(1,-1)
        
        if self.stats["num_elements"] == 0:
            self.stats["mean"] = mean_vals
            self.stats["var"] = var_vals
            self.stats["min"] = min_vals
            self.stats["max"] = max_vals
            self.stats["num_elements"] = num_elements

            return

        curr_mean = self.stats["mean"].clone()
        self.stats["mean"] = (self.stats["mean"] * self.stats["num_elements"] + mean_vals * num_elements) / (
            self.stats["num_elements"] + num_elements
        )
        self.stats["var"] = self.update_var(
            self.stats["var"],
            self.stats["num_elements"],
            var_vals,
            num_elements,
            curr_mean,
            mean_vals,
            self.stats["mean"],
        )
        self.stats["min"] = torch.min(self.stats["min"], min_vals)
        self.stats["max"] = torch.max(self.stats["max"], max_vals)
        self.stats["num_elements"] += num_elements

    @staticmethod    
    def update_var(curr_var,num_elem,new_var,new_num_elem,
                   curr_mean,new_mean,updated_mean):

        numerator = ((num_elem - 1) * curr_var + (new_num_elem - 1) * new_var) + num_elem * (curr_mean - updated_mean) ** 2 + new_num_elem * (new_mean - updated_mean) ** 2
        return numerator / (num_elem + new_num_elem - 1)
    
    @classmethod
    def add_as_pre_forward_hook(cls,layer,per_channel=True,device="cpu",num_channels = 1):
        stats = cls(per_channel,device,num_channels,name = layer.full_name)
        layer.stats = stats
        handle = layer.register_forward_pre_hook(layer.stats.update)
        return handle


In [126]:
type(model.model.diffusion_model)#.input_blocks
unet = model.model.diffusion_model

In [145]:
len(unet.input_blocks)

12

In [146]:
h = []
for b in unet.input_blocks:
    print(b.full_name)
    b._forward_pre_hooks.clear()
    h.append(DynamicStats.add_as_pre_forward_hook(b,per_channel=True))
    

input_blocks.0
input_blocks.1
input_blocks.2
input_blocks.3
input_blocks.4
input_blocks.5
input_blocks.6
input_blocks.7
input_blocks.8
input_blocks.9
input_blocks.10
input_blocks.11


In [142]:
ts[0].shape #.keys()#['xs'].keys()

torch.Size([6400, 4, 64, 64])

In [151]:
unet(ts[0][1:2].cuda(),ts[1][1:2].cuda(),ts[2][1:2].cuda())

tensor([[[[ 1.0197, -1.2224,  0.8171,  ...,  0.6459, -1.1296,  0.3178],
          [-1.7219,  0.7103, -0.3764,  ..., -0.8568,  0.2571, -0.2698],
          [-1.0704, -0.6643, -0.9427,  ...,  0.0974, -0.3038, -0.3247],
          ...,
          [-2.0598, -0.4508, -0.2458,  ..., -0.4007,  0.7685,  0.3434],
          [-1.1189,  0.3298,  0.2467,  ..., -1.3688,  0.7493, -1.2775],
          [ 0.4450,  0.5930,  1.2368,  ..., -0.4862,  1.2466, -1.3451]],

         [[-0.7428,  1.0800,  0.5523,  ..., -0.5421, -0.2681,  0.4057],
          [-0.9825,  0.1489, -1.4920,  ...,  0.3289, -0.7617, -2.4848],
          [ 0.1387,  0.3688, -0.3063,  ..., -1.3391, -0.2750,  0.7139],
          ...,
          [ 2.0237, -0.9864, -1.5527,  ..., -0.9398, -0.3191,  1.1595],
          [-0.6533, -0.4209,  0.5812,  ...,  0.4143, -1.0974,  1.0536],
          [-0.4780, -1.3158, -0.2471,  ..., -0.0352,  2.3882, -0.5478]],

         [[ 0.2956,  0.2108,  0.1432,  ..., -0.8663, -1.1519, -1.3468],
          [-0.3990, -0.7966, -

In [154]:
unet.input_blocks[11].stats.stats

{'mean': tensor([[-0.4137,  0.0907,  0.5297,  ..., -0.0279, -1.2098,  0.2225]],
        grad_fn=<DivBackward0>),
 'var': tensor([[4.0080, 2.0733, 1.4303,  ..., 4.4712, 4.5218, 2.0721]],
        grad_fn=<DivBackward0>),
 'min': tensor([[-5.5618, -3.4583, -2.3769,  ..., -4.1596, -5.7859, -3.5346]],
        grad_fn=<MinimumBackward0>),
 'max': tensor([[4.4649, 4.4506, 5.0405,  ..., 7.4011, 3.5064, 3.7479]],
        grad_fn=<MaximumBackward0>),
 'num_elements': 128}

In [155]:
unet.input_blocks[11].stats.stats['mean'].shape


torch.Size([1, 1280])

In [88]:
n_x,c = 10,15
var_x, mean_x =  4.0, 5.0
x = torch.randn(n_x,c,3,4) * np.sqrt(var_x) + mean_x
x.shape

torch.Size([10, 15, 3, 4])

In [70]:
data = x.permute(1, 0, 2, 3).contiguous().view(c, n_x * 3 * 4)
data.shape

torch.Size([15, 120])

In [105]:
#data.max(dim=0)[0].reshape(1,-1).shape
data.var(dim=1)#.reshape(1,-1).shape

tensor([4.6144, 3.2301, 3.4406, 4.4639, 4.9173, 3.8344, 3.5392, 4.3008, 3.3813,
        3.3667, 3.3924, 3.3153, 2.9279, 4.0698, 4.0251])

In [108]:
ds = DynamicStats(per_channel=True,num_channels=c)

In [89]:
ds.update(x)

In [109]:
for i in range(1000):
    var_x, mean_x =  4.0, 5.0
    x = torch.randn(n_x,c,3,4) * np.sqrt(var_x) + mean_x
    ds.update(x)

In [110]:
ds.stats["var"].shape

torch.Size([1, 15])

In [111]:
ds.stats['num_elements'],ds.stats["var"],ds.stats["mean"]

(120000,
 tensor([[4.0098, 3.9901, 4.0118, 3.9893, 3.9960, 3.9894, 4.0020, 3.9826, 4.0125,
          3.9930, 3.9950, 3.9625, 3.9925, 4.0109, 3.9775]]),
 tensor([[5.0023, 5.0043, 5.0037, 5.0016, 4.9979, 4.9927, 5.0004, 4.9961, 4.9943,
          5.0027, 4.9945, 4.9987, 5.0007, 5.0039, 5.0038]]))

In [47]:
std_x, mean_x, n_x = np.sqrt(4.0), 5.0, 10000  # std, mean, and count for dataset x
std_y, mean_y, n_y = np.sqrt(9.0), 10.0, 2500  # std

x = torch.randn(n_x) * std_x + mean_x
y = torch.randn(n_y) * std_y + mean_y

In [48]:
torch.var(x), torch.mean(x), torch.var(y), torch.mean(y)

(tensor(4.0073), tensor(5.0045), tensor(9.4503), tensor(9.8960))

In [49]:
xy = torch.cat([x, y])
xy.size()

torch.Size([12500])

In [50]:
torch.var(xy), torch.mean(xy)

(tensor(8.9239), tensor(5.9828))

In [51]:
var_x = torch.var(x) 
var_y = torch.var(y) 
mean_xy = torch.mean(xy)

numerator = ((n_x - 1) * var_x + (n_y - 1) * var_y) + n_x * (mean_x - mean_xy) ** 2 + n_y * (mean_y - mean_xy) ** 2
denominator = n_x + n_y - 1

(numerator / denominator)

tensor(9.0959)

In [182]:
def get_train_samples(sample_data, num_samples=128,timesteps=None):
        # get the real number of timesteps (especially for DDIM)
    if timesteps is None:
        nsteps = len(sample_data["xs"])
        num_st = nsteps // 2
        timesteps = list(range(0, nsteps, nsteps//num_st))
    #logger.info(f'Selected {len(timesteps)} steps from {nsteps} sampling steps')
    xs_lst = [sample_data["xs"][i][:num_samples] for i in timesteps]
    ts_lst = [sample_data["ts"][i][:num_samples] for i in timesteps]
    if True:#args.cond:
        xs_lst += xs_lst
        ts_lst += ts_lst
        conds_lst = [sample_data["cs"][i][:num_samples] for i in timesteps] + [sample_data["ucs"][i][:num_samples] for i in timesteps]
    xs = torch.cat(xs_lst, dim=0)
    ts = torch.cat(ts_lst, dim=0)
   
    conds = torch.cat(conds_lst, dim=0)
    return xs, ts, conds
    

In [165]:
#calib_data_path = "/fastdata/users/nadavg/sd/qdiff/sd_coco-s75_sample1024_allst.pt"
#calib_data_path = "/fastdata/users/nadavg/sd/qdiff/gen_calib/calib_dict_steps50.pt"
calib_data_path = "/fastdata/users/nadavg/sd/qdiff/gen_calib/calib_dict_steps20.pt"

In [166]:
sample_data = torch.load(calib_data_path)

/tmp/ipykernel_3482357/822771670.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sample_data = torch.load(calib_data_path)


In [176]:
#sample_data.keys()
sample_data['ts'].shape

torch.Size([20, 256])

In [169]:
sample_data['ts'][-1][0]

tensor(1)

In [183]:
ts = get_train_samples(sample_data=sample_data, num_samples=11,timesteps=[0,1])

In [184]:
ts[0].shape,ts[1].shape,ts[2].shape

(torch.Size([44, 4, 64, 64]), torch.Size([44]), torch.Size([44, 77, 768]))

In [185]:
11*2*2

44

In [175]:
4*10

40

In [29]:
import torch
import torch.nn as nn

# Define a simple layer that modifies input
class CustomLayer(nn.Module):
    def forward(self, x):
        x += 1  # Modifying input inside forward
        return x

# Hook functions
def pre_forward_hook(module, input):
    print(f"Pre-Forward Hook: Input = {input}")

def forward_hook(module, input, output):
    print(f"Forward Hook: Input = {input}, Output = {output}")

# Instantiate model and register hooks
layer = CustomLayer()
layer.register_forward_pre_hook(pre_forward_hook)
layer.register_forward_hook(forward_hook)

# Example input
x = torch.tensor([2.0])

# Forward pass
print("Running forward pass...\n")
output = layer(x)


Running forward pass...

Pre-Forward Hook: Input = (tensor([2.]),)
Forward Hook: Input = (tensor([3.]),), Output = tensor([3.])
